In [1]:
import numpy as np
import QuantLib as ql
import csv
import random
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [2]:
today = ql.Date(1,1,2022)
ql.Settings.instance().setEvaluationDate(today)
day_counter = ql.Actual365Fixed()

In [3]:
def fdm(strike, forward, tenor, alpha, beta, nu, rho, t_grid=20, f_grid=200, x_grid=25, scaling_factor=1.0, eps=.0001):
    maturity_date = today + ql.Period(int(round(365 * tenor)), ql.Days)
    r_ts = ql.YieldTermStructureHandle(ql.FlatForward(today, 0.0, day_counter))
    vol_ts = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(0, ql.NullCalendar(), 0.0, day_counter))
    bs_process = ql.GeneralizedBlackScholesProcess(ql.QuoteHandle(ql.SimpleQuote(forward)), r_ts, r_ts, vol_ts)
    option = ql.VanillaOption(ql.PlainVanillaPayoff(ql.Option.Call, strike), ql.EuropeanExercise(maturity_date))
    option.setPricingEngine(ql.FdSabrVanillaEngine(forward, alpha, beta, nu, rho, r_ts, t_grid, f_grid, x_grid, 0, scaling_factor, eps))
    return option.impliedVolatility(option.NPV(), bs_process, 1.0e-4, 1000, 1.0e-5, 1.0)

In [4]:
file_name = 'vol_data/fdm_hagan_vol_train2.csv'

# "Strike" and "Nu" here are transformed parameters.

# "Strike" is an approximation for the number of std devs up or down w.r.t. ATM vol, assuming beta = 0.
# "Strike_0" = e^(Strike * Alpha * sqrt(Tenor)) is the actual scaled strike price we use for computing imp. vol.
# The effect of this is that we consider larger strike ranges for longer tenors and larger ATM vols.

# "Nu" is the volvol scaled by the square root of time.
# "Nu_0" = Nu / sqrt(Tenor) is the actual volvol we use for computing imp. vol.
# The effect of this is that we consider smaller volvol ranges for longer tenors.

strikes = [-2, -1.875, -1.75, -1.625, -1.5, -1.25, -1.125, -1, -.875, -.75, -.5, -.25, 0, .25, .5, .75, 0.875, 1, 1.125, 1.25, 1.5, 1.625, 1.75, 1.875, 2]
tenors = [0.5, 1, 1.5, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 16, 18, 21, 24, 27, 30]
alphas = [0.001, 0.005, .01, .02, 0.025, .03, .04, 0.045, .05, .06, 0.065, .07, .08, 0.085, .09]
betas = [.001, 0.005, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
nus = [.1, .5, 0.75, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 4.75, 5]
rhos = [-.4, -.35, -.3, -.25, -.2, -.15, -.1, -.05, 0, .05, .1, .15, .2, .25, .3, .35, .4]

pstrike = [0.05,0.05,0.05,0.05,0.04,0.04,0.04,0.04,0.04,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.04,0.04,0.04,0.05,0.05,0.05,0.05]
ptenor = [0.07,0.07,0.06,0.05,0.04,0.04,0.04,0.04,0.04,0.05,0.05,0.04,0.04,0.04,0.04,0.04,0.05,0.06,0.07,0.07]

with open(file_name, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Strike", "Tenor", "Alpha", "Beta", "Nu", "Rho", "Strike_0", "Nu_0", "FDM_vol", "Hagan_vol"])

keys = set()
for i in tqdm(range(100)):
    with open(file_name, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        while len(keys) < i * 1000:
            strike = np.random.choice(strikes,size=1,p=pstrike)[0]
            tenor = np.random.choice(tenors,size=1,p=ptenor)[0]
            alpha = random.choice(alphas)
            beta = random.choice(betas)
            nu = random.choice(nus)
            rho = random.choice(rhos)
            strike_0 = np.exp(strike * alpha * tenor**.5)
            nu_0 = nu / tenor**.5
            if (strike, tenor, alpha, beta, nu, rho) not in keys:
                keys.add((strike, tenor, alpha, beta, nu, rho))
                try:
                    fdm_vol = fdm(strike_0, 1, tenor, alpha, beta, nu_0, rho, 50, 800, 40, eps=.00001)
                    if fdm_vol != 1.0e-5:
                        writer.writerow([
                            strike,
                            tenor,
                            alpha,
                            beta,
                            nu,
                            rho,
                            strike_0,
                            nu_0,
                            fdm_vol,
                            ql.sabrVolatility(strike_0, 1, tenor, alpha, beta, nu_0, rho)
                        ])
                except:
                    pass

/var/folders/hp/vvh9r6ts7wg9d4vngzgrz1km0000gn/T/ipykernel_11465/1581513931.py:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(100)):


  0%|          | 0/100 [00:00<?, ?it/s]

In [5]:
import pandas as pd
kk = pd.read_csv('vol_data/fdm_hagan_vol_train2.csv')
kk

,Strike,Tenor,Alpha,Beta,Nu,Rho,Strike_0,Nu_0,FDM_vol,Hagan_vol
0,-1.250,7.0,0.065,0.010,0.50,0.35,0.806568,0.188982,0.070241,0.070030
1,1.250,9.0,0.050,0.150,0.10,0.40,1.206230,0.033333,0.047525,0.047537
2,1.625,21.0,0.070,0.250,0.75,0.10,1.684162,0.163663,0.076151,0.077786
3,1.625,9.0,0.001,0.300,2.50,0.25,1.004887,0.833333,0.002016,0.003074
4,-0.250,16.0,0.085,0.001,0.50,-0.05,0.918512,0.125000,0.091401,0.091451
...,...,...,...,...,...,...,...,...,...,...
91635,1.875,1.5,0.005,0.400,2.50,0.40,1.011548,2.041241,0.010856,0.016409
91636,-1.875,24.0,0.025,0.001,5.00,-0.05,0.794820,1.020621,0.045732,0.259051
91637,0.500,27.0,0.090,0.010,3.50,0.15,1.263426,0.673575,0.084213,0.231609
91638,-1.625,24.0,0.050,0.150,0.50,0.35,0.671634,0.102062,0.057786,0.057607


In [ ]:
file_name = 'vol_data/fdm_hagan_vol_test.csv'

with open(file_name, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Strike", "Tenor", "Alpha", "Beta", "Nu", "Rho", "Strike_0", "Nu_0", "FDM_vol", "Hagan_vol"])

for i in range(5):
    with open(file_name, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for j in range(10000):
            strike = random.uniform(-2, 2)
            tenor = random.uniform(.5, 30)
            alpha = random.uniform(.001, .09)
            beta = random.uniform(.001, .5)
            nu = random.uniform(.1, 5)
            rho = random.uniform(-.4, .4)
            strike_0 = np.exp(strike * alpha * tenor**.5)
            nu_0 = nu / tenor**.5
            try:
                fdm_vol = fdm(strike_0, 1, tenor, alpha, beta, nu_0, rho, 50, 800, 40, eps=.00001)
                if fdm_vol != 1.0e-5:
                    writer.writerow([
                        strike,
                        tenor,
                        alpha,
                        beta,
                        nu,
                        rho,
                        strike_0,
                        nu_0,
                        fdm_vol,
                        ql.sabrVolatility(strike_0, 1, tenor, alpha, beta, nu_0, rho)
                    ])
            except:
                pass

In [ ]:
f = 100
T = 2
alpha = 0.02
beta = 0.2
nu = 2.0
rho = 0.2
strike_range = np.linspace(-1, 1, 100)

In [ ]:
f = 200
T = 15
alpha = 0.06
beta = 0.5
nu = 2.0
rho = -0.3
strike_range = np.linspace(-1.75, 1.75, 100)

In [ ]:
f = 200
strike = 0
alpha = 0.06
beta = 0.3
nu = 3
rho = -0.3
time_range = np.linspace(0.5, 30, 100)